In [5]:
"""Readme :  pip install python-telegram-bot --upgrade"""

import logging
from typing import Dict

from telegram import ReplyKeyboardMarkup, Update, ReplyKeyboardRemove
from telegram.ext import (
    Updater,
    CommandHandler,
    MessageHandler,
    Filters,
    ConversationHandler,
    CallbackContext,
)

# Enable logging
logging.basicConfig(
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s', level=logging.INFO
)
TOKEN='5389959641:AAFzapxb0HqNfNtEKCBrpVJzTx5AexVX5Bk'

# State definitions for top level conversation
CHOOSING = map(chr, range(1))
# Shortcut for ConversationHandler.END
END = ConversationHandler.END

reply_keyboard = [
    [ 'Search movie by plot events','Recommend movies by similarity'],]
markup = ReplyKeyboardMarkup(reply_keyboard, one_time_keyboard=True)



def start(update: Update, context: CallbackContext) -> int:
    """Start the conversation and ask user for input."""
    update.message.reply_text(
        
        "Hi  ! I'm Moviezam! I know everything about movies! \n" 
        "Do you want to search for movies with a particular plot event? \n" 
        "Can i suggest you similar movies to the ones you like?\n" 
        "Choose on of the options below, I start searching for you\n"
         "To abort, simply type /stop.",
        reply_markup=markup,
    )

    return CHOOSING

def answer_recommend(update: Update, context: CallbackContext) -> int:
    """Ask the user for info about the selected predefined choice."""
    text = update.message.text
    context.user_data['choice'] = text
    update.message.reply_text('Please write the title of a movie that you like, i will suggest you 5 similar movies')

    return TYPING_REPLY

def answer_plot(update: Update, context: CallbackContext) -> int:
    """Ask the user for info about the selected predefined choice."""
    text = update.message.text
    context.user_data['choice'] = text
    update.message.reply_text('Please write some plot events or details and i will show you some related movies')

    return TYPING_REPLY

def stop(update: Update, context: CallbackContext) -> int:
    """End Conversation by command."""
    update.message.reply_text('Okay, bye.')

    return END


def main() -> None:
    """Run the bot."""
    # Create the Updater and pass it your bot's token.
    updater = Updater(TOKEN)

    # Get the dispatcher to register handlers
    dispatcher = updater.dispatcher

    # Add conversation handler with the states CHOOSING, TYPING_CHOICE and TYPING_REPLY
    conv_handler = ConversationHandler(
        entry_points=[CommandHandler('start', start)],
        states={
            CHOOSING: [
                MessageHandler(
                    Filters.regex('^(Search movie by plot events)$'), answer_plot   ),
                MessageHandler(Filters.regex('^Recommend movies by similarity$'), answer_recommend),
            ],
       
        },
        
        fallbacks=[CommandHandler('stop', stop)],
    )

    dispatcher.add_handler(conv_handler)

    # Start the Bot
    updater.start_polling()

    # Run the bot until you press Ctrl-C or the process receives SIGINT,
    # SIGTERM or SIGABRT. This should be used most of the time, since
    # start_polling() is non-blocking and will stop the bot gracefully.
    updater.idle()


if __name__ == '__main__':
    main()


2022-04-24 22:00:43,073 - apscheduler.scheduler - INFO - Scheduler started
2022-04-24 22:00:55,082 - telegram.ext.dispatcher - ERROR - No error handlers are registered, logging exception.
Traceback (most recent call last):
  File "C:\Users\cioffil\Desktop\DS\Github\Telegram_chatbots\venv\lib\site-packages\telegram\ext\dispatcher.py", line 555, in process_update
    handler.handle_update(update, self, check, context)
  File "C:\Users\cioffil\Desktop\DS\Github\Telegram_chatbots\venv\lib\site-packages\telegram\ext\conversationhandler.py", line 626, in handle_update
    new_state = handler.handle_update(update, dispatcher, check_result, context)
  File "C:\Users\cioffil\Desktop\DS\Github\Telegram_chatbots\venv\lib\site-packages\telegram\ext\handler.py", line 198, in handle_update
    return self.callback(update, context)
  File "C:\Users\cioffil\AppData\Local\Temp\ipykernel_16728\3859771118.py", line 61, in answer_plot
    return TYPING_REPLY
NameError: name 'TYPING_REPLY' is not defined
2

In [9]:
import pandas as pd

In [13]:
!pip install openpyxl

  Using cached openpyxl-3.0.9-py2.py3-none-any.whl (242 kB)
  Using cached et_xmlfile-1.1.0-py3-none-any.whl (4.7 kB)


You should consider upgrading via the 'C:\Users\cioffil\Desktop\DS\Github\Telegram_chatbots\venv\Scripts\python.exe -m pip install --upgrade pip' command.


In [14]:
data_movielens_final_new=pd.read_excel('C:/Users/cioffil/Desktop/DS/Github/Moviezam/Moviezam/data/movielens/data_movielens_final_new.xlsx')

In [15]:
data_movielens_final_new.head()

,Unnamed: 0,ID,text,text_clean,Title,original_title,votes_score,year,languages,overview,plot_outline,Synopsis
0,0,114709,"Led by Woody, Andy's toys live happily in his ...",lead woody andy toy live happily room andy bir...,Toy Story,Toy Story,0.933155,1995.0,['English'],"Led by Woody, Andy's toys live happily in his ...",A little boy named Andy loves to be in his roo...,A boy called Andy Davis (voice: John Morris ) ...
1,1,113497,When siblings Judy and Peter discover an encha...,sibling judy peter discover enchant board game...,Jumanji,Jumanji,0.861954,1995.0,"['English', 'French']",When siblings Judy and Peter discover an encha...,"Jumanji, one of the most unique--and dangerous...",The film begins in 1869 in the town of Brantfo...
2,2,113497,Alan's only hope for freedom is to finish the ...,alan hope freedom finish game prove risky thre...,Jumanji,Jumanji,0.861954,1995.0,"['English', 'French']",When siblings Judy and Peter discover an encha...,"Jumanji, one of the most unique--and dangerous...",The film begins in 1869 in the town of Brantfo...
3,3,113228,A family wedding reignites the ancient feud be...,family wed reignite ancient feud next door nei...,Grumpier Old Men,Grumpier Old Men,0.691204,1995.0,"['English', 'Italian', 'German']",A family wedding reignites the ancient feud be...,Things don't seem to change much in Wabasha Co...,NaN
4,4,113228,"Meanwhile, a sultry Italian divorcée opens a r...",meanwhile sultry italian divorcée open restaur...,Grumpier Old Men,Grumpier Old Men,0.691204,1995.0,"['English', 'Italian', 'German']",A family wedding reignites the ancient feud be...,Things don't seem to change much in Wabasha Co...,NaN
